# 🛠️ Notebook 06: Feature Engineering

**Autor:** Gian  
**Fecha:** 2026-01-19  
**Objetivo:** Crear nuevas variables predictivas (features) para mejorar el rendimiento del modelo.

---

## 📋 Contenido

1. Configuración del entorno
2. Carga de datos
3. Creación de Features de Interacción
4. Creación de Features Financieras (Ratios)
5. Transformaciones Matemáticas (Log)
6. Features Temporales y de Comportamiento
7. Encoding y Escalado del Dataset Enriquecido
8. Guardado y Reporte

---
## 1. Configuración del Entorno

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

# Seed para reproducibilidad
np.random.seed(42)

print("✅ Librerías importadas correctamente")

---
## 2. Carga de Datos
Cargamos `data_clean.csv` (No el transformado) para tener los valores originales y poder crear ratios con sentido de negocio.

In [ ]:
# Rutas
OUTPUT_PATH = Path("../../outputs/gian")
CLEAN_DATA_PATH = OUTPUT_PATH / "data" / "data_clean.csv"
ENGINEERED_DATA_PATH = OUTPUT_PATH / "data"
REPORTS_PATH = OUTPUT_PATH / "reports"

# Cargar datos limpios
df = pd.read_csv(CLEAN_DATA_PATH)

print(f"✅ Dataset base cargado: {df.shape[0]:,} registros × {df.shape[1]} columnas")

---
## 3. Creación de Features de Interacción
Basado en el EDA, combinamos variables fuertemente correlacionadas con el riesgo.

In [ ]:
df_fe = df.copy()

# 1. Interacción Riesgo-Soporte
# Hipótesis: Clientes riesgosos que contactan mucho a soporte son críticos
df_fe['riesgo_soporte_interaccion'] = df_fe['score_riesgo'] * df_fe['tickets_soporte']

# 2. Intensidad de Uso
# Hipótesis: Conexiones por día activo da una mejor idea de la intensidad de uso
df_fe['intensidad_uso'] = df_fe['conexiones_mensuales'] / (df_fe['dias_activos_semanales'] * 4 + 1) # +1 para evitar div/0

print("✅ Features de interacción creadas: riesgo_soporte_interaccion, intensidad_uso")

---
## 4. Creación de Features Financieras (Ratios)
Ratios que indican la "carga" financiera para el cliente o el valor del cliente para la empresa.

In [ ]:
# 1. Ratio Cargo / Ingreso (Affordability)
# Qué porcentaje del ingreso representa el servicio
df_fe['ratio_carga_financiera'] = df_fe['cargo_mensual'] / (df_fe['ingresos_totales'] + 1)

# 2. Valor Promedio Diario
df_fe['valor_diario_cliente'] = df_fe['cargo_mensual'] / 30

print("✅ Features financieras creadas: ratio_carga_financiera, valor_diario_cliente")

---
## 5. Transformaciones Matemáticas (Log)
Aplicamos Logaritmo a variables con alto sesgo (Skewness > 1) detectadas en el EDA.

In [ ]:
skewed_vars = ['errores_pago', 'tickets_soporte', 'antiguedad']

for var in skewed_vars:
    # log1p calcula log(1 + x) para manejar ceros
    df_fe[f'log_{var}'] = np.log1p(df_fe[var])

print(f"✅ Transformación Log aplicada a: {skewed_vars}")

---
## 6. Encoding y Escalado
Ahora aplicamos las transformaciones finales al dataset enriquecido.

In [ ]:
# Separar columnas numéricas y categóricas
categorical_cols = df_fe.select_dtypes(include=['object']).columns.tolist()
exclude_cols = ['cliente_id', 'fecha_registro', 'fecha_ultimo_pago', 'pais', 'ciudad']
categorical_cols = [c for c in categorical_cols if c not in exclude_cols]

# 1. One-Hot Encoding
df_encoded = pd.get_dummies(df_fe, columns=categorical_cols, drop_first=True)

# 2. Scaling (StandardScaler) para todas las numéricas (excepto IDs y target)
numeric_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [c for c in numeric_cols if c not in ['cliente_id', 'cancelacion']]

scaler = StandardScaler()
df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

print("✅ Encoding y Scaling aplicados correctamente")
print(f"📊 Shape Final: {df_encoded.shape}")

---
## 7. Guardado y Reporte

In [ ]:
# Guardar Dataset Engineering
output_file = ENGINEERED_DATA_PATH / "06_data_engineered.csv"
df_encoded.to_csv(output_file, index=False)

# Generar Reporte
report = {
    'Original_Columns': df.shape[1],
    'New_Features_Created': df_encoded.shape[1] - df.shape[1], # Aproximado por OHE
    'Final_Columns': df_encoded.shape[1],
    'New_Interaction_Features': ['riesgo_soporte_interaccion', 'intensidad_uso'],
    'New_Financial_Features': ['ratio_carga_financiera', 'valor_diario_cliente'],
    'Log_Transformed': skewed_vars
}

report_df = pd.DataFrame([str(v) for v in report.values()], index=report.keys(), columns=['Value'])
report_df.to_csv(REPORTS_PATH / "06_feature_engineering_report.csv")

print(f"💾 Dataset guardado en: {output_file}")
print("📋 Reporte generado.")